In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras.utils import Sequence
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import Adam
import time
import pickle

Using TensorFlow backend.


#### If using AMD GPU, switch backend to PlaidML library:

In [ ]:
import os
os.environ['KERAS_BACKEND']='plaidml.keras.backend'

# When using plaidml, the libraries are imported from keras instead of tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

#### Define Experiments

In [2]:
conv_layers = [1,3,5]      # number of conv layers
layer_sizes = [32,64,128]  # number of nodes in a layer
dense_layers = [1,2,3]     # number of dense layers

#### Load input data

In [3]:
pickle_in = open('../Dataset/df_10pct.pickle', 'rb')
df_train, df_test = pickle.load(pickle_in)

In [4]:
# The Keras ImageDataGenerator uses string type data label
df_train['gender'] = df_train.gender.astype(str)
df_test['gender'] = df_test.gender.astype(str)

In [5]:
print(df_train.shape, df_test.shape)

(42965, 10) (2261, 10)


In [6]:
df_train.dtypes

path                object
id                  uint16
name                object
dob         datetime64[ns]
gender              object
score1             float64
score2             float64
pic_date    datetime64[ns]
region              object
age                float64
dtype: object

We will be using a generator to feed model with images, the X would be the path to these images. y will be the gender label.

In [7]:
image_reshape_size = 120
input_image_root_dir = '../Dataset/imdb_crop/' # Don't forget the ending slash

In [8]:
from keras import backend as K
K.set_image_data_format('channels_last')
batch_size = 64
inputShape = (image_reshape_size, image_reshape_size, 1)

In [9]:
df_train.head(3)

,path,id,name,dob,gender,score1,score2,pic_date,region,age
120097,12/nm0001612_rm2398793472_1969-8-19_2012.jpg,13149,Matthew Perry,1969-08-19,1,0.799562,NaN,2012-01-01,"[397.6, 52.0, 454.4, 108.8]",42.369111
110887,35/nm0001435_rm3857046784_1963-7-30_1994.jpg,11966,Lisa Kudrow,1963-07-30,0,0.774393,NaN,1994-01-01,"[1132.544, 313.344, 1277.952, 458.752]",30.426361
59955,59/nm0000459_rm3615721728_1960-8-16_2008.jpg,18968,Timothy Hutton,1960-08-16,1,2.119279,NaN,2008-01-01,"[282.03335656952333, 55.67480128185821, 342.96...",47.376743


#### Set up input image generator using flow_from_dataframe

In [10]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="training",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=32,
                                            seed=1,
                                            shuffle=True)

val_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="validation",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=32,
                                            seed=1,
                                            shuffle=True)

test_generator = datagen.flow_from_dataframe(dataframe=df_test, 
                                            directory=input_image_root_dir, 
                                            x_col="path", y_col=None, 
                                            class_mode=None, 
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=1,
                                            shuffle=False)

Found 38669 images belonging to 2 classes.
Found 4296 images belonging to 2 classes.
Found 2261 images.


#### (1) Run training experiments

In [ ]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            NAME = 'BN-{}-conv-{}-node-{}-dens-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))  # model name with timestamp
            print(NAME) 
            
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            callbacks = [tensorboard]
            
            model = Sequential()
            
            # first layer
            model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu", input_shape=inputShape))
            model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=(3,3)))
            
            # sets up additional # of conv layers
            for _ in range(conv_layer - 1):
#                 layer_size *= 2
                model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
#                 model.add(BatchNormalization())
#                 model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
#                 model.add(BatchNormalization())
                model.add(MaxPooling2D(pool_size=(2,2)))
                model.add(Dropout(0.25))
            
            model.add(Flatten())
            
            layer_size *= 4 # to get the dense layer to be 8X of last output size
            
            # sets up # of dense layers
            for _ in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
#                 model.add(BatchNormalization())
#                 model.add(Dropout(0.5))
            
            # output layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            opt = Adam(lr=0.001)
            model.compile(loss='binary_crossentropy', 
                          optimizer=opt,
                          metrics=['accuracy'])

            model.fit_generator(generator=train_generator,
                                steps_per_epoch=(train_generator.n // train_generator.batch_size),
                                callbacks = callbacks,
                                validation_data=val_generator,
                                validation_steps=(val_generator.n // val_generator.batch_size),
                                epochs=30,
                                use_multiprocessing=False,
                                workers=1)

BN-5-conv-32-node-2-dens-1553790061
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
1208/1208 [==============================] - 1238s 1s/step - loss: 0.7853 - acc: 0.5874 - val_loss: 0.6441 - val_acc: 0.6413
Epoch 2/30
1208/1208 [==============================] - 1232s 1s/step - loss: 0.6858 - acc: 0.6276 - val_loss: 0.6666 - val_acc: 0.6348
Epoch 3/30
1208/1208 [==============================] - 1208s 1s/step - loss: 0.6639 - acc: 0.6618 - val_loss: 0.5909 - val_acc: 0.7000
Epoch 4/30
1208/1208 [==============================] - 1209s 1s/step - loss: 0.6468 - acc: 0.6792 - val_loss: 0.6062 - val_acc: 0.6935
Epoch 5/30
1208/1208 [==============================] - 1224s 1s/step - loss: 0.6352 - acc: 0.6893 - val_loss: 0.6245 - val_acc: 0.6639
Epoch 6/30
1208/1208 [===========================

#### VGG-16 model

In [ ]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Lambda, Conv2D
from keras.layers import GlobalAveragePooling2D, Input, Dropout
from keras.layers.convolutional import MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, SGD
import numpy as np
import urllib
# from keras.utils import plot_model

model_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
target_size=(224, 224)
batch_size=32

def preprocess_image(im):
    vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
    #im = cv2.resize(cv2.imread(path), (224, 224)).astype(np.float32)
    im = (im - vgg_mean)
    return im[:, ::-1] # RGB to BGR


model = Sequential()

model.add(Lambda(preprocess_image, input_shape=inputShape))

model.add(Conv2D(64, (3, 3), input_shape=inputShape, activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1000, activation='softmax'))

model.load_weights(model_path)


x = Dense(1, activation='softmax')(model.layers[-2].output)
model = Model(model.input, x)

for layer in model.layers: 
    layer.trainable=False

opt = Adam(lr=0.00001)
model.compile(optimizer=opt,
            loss='binary_crossentropy', 
            metrics=['accuracy'])

model.fit_generator(generator=train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size,
                validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size)


# for layer in model.layers[:10]:
#     layer.trainable = False
# for layer in model.layers[10:]:
#     layer.trainable = True

# opt = SGD(lr=10e-5)
# model.compile(optimizer=opt,
#               loss='categorical_crossentropy', 
#               metrics=['accuracy'])

# model.fit_generator(generator=train_generator, steps_per_epoch=batches.samples//batch_size, nb_epoch=1,
#                 validation_data=valid_generator, validation_steps=valid_batches.samples//batch_size)

To view the tensorboard, use command:
tensorboard --logdir=logs/

#### Evaluation

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=test_generator.n//test_generator.batch_size,
                            verbose=1)
pred_class=np.argmax(pred,axis=1) # index of largest value in each row